In [ ]:
WITH Canadian_PEP AS (
    -- Prepare the Reference Table: Filter for Canada and extract the raw Cost Center ID
    SELECT 
        EmployeeName,
        ACIF,
        `Title/Department`,
        Costcenter AS Original_PEP_CostCenter,
        PeopleManager,
        -- Grabs everything before the first space ("0533 OTTAWA..." -> "0533" | "ZHAH2" -> "ZHAH2")
        SUBSTRING_INDEX(TRIM(Costcenter), ' ', 1) AS Extracted_CC
    FROM hive_metastore.ra_adido_2025.employee_pep_list_as_of_oct312025
    WHERE TRIM(Region) = 'Canada'
)

-- Read each row from the Mapping File and attach PEP details if they exist
SELECT 
    -- 1. Base Columns from the Mapping File (These will ALWAYS populate)
    m.`Cost Center` AS Mapping_Col_A_CC,
    m.`Assessable Unit ID and Name` AS Mapping_Col_B_AU,
    m.Segment AS Mapping_Col_C_Segment,
    m.`Additional Assessable Unit ID and Name` AS Mapping_Col_D_Addtl_AU,
    m.Segment2 AS Mapping_Col_E_Segment2,
    m.`Contact of the mapping Confirmation` AS Mapping_Col_F_Contact,
    
    -- 2. Reference Columns from the PEP List (These will be NULL if no officer is found)
    p.EmployeeName,
    p.ACIF,
    p.`Title/Department`,
    p.Original_PEP_CostCenter
    
FROM cost_center_mapping m
LEFT JOIN Canadian_PEP p
    -- Smart Join: If the mapping ID is exactly 3 characters (e.g., '533'), add a '0' to the front.
    -- Otherwise, leave it completely alone (so 'ZHAH2' or '1967' don't get altered or truncated).
    ON CASE 
        WHEN LENGTH(TRIM(CAST(m.`Cost Center` AS STRING))) = 3 
        THEN CONCAT('0', TRIM(CAST(m.`Cost Center` AS STRING))) 
        ELSE TRIM(CAST(m.`Cost Center` AS STRING)) 
       END = p.Extracted_CC